# Best Practices: Control Flow

Control flow statements (``if``, ``for``, ``while``, etc blocks) make decisions based on conditions and execute either certain lines of code or execute a given number of times. 

Control flow can become convoluted quickly if guidelines are not kept in mind. Control blocks themselves add a bit of complexity to your code. Instead of simply reading line-by-line, the reader/programmer must try to follow several cases or decide when the loop has accomplished its purpose. This is increasing complexity, not reducing it.

### **Use a well-named variable for non-obvious conditions**.
This was mentioned in the variables section, but is worth repeating here.

It's easy to write a statement like:

```python
if process_num < 10 and monte_carlo_runs > 100:
    ...
```
At the time of writing, it's obvious what you're testing, but rereading it later it will not be obvious.
Instead, use variables that describe the condition you're really testing:
```python
# the actual conditions are moved to a boolean statement
too_many_runs_queued = process_num < 10 and monte_carlo_runs > 100

# now the real meaningful condition is compared, not the individual conditions
if too_many_runs_queued:
    # reduce runs
    ...
```

### **Make the most common scenario the first case.**

When reading and writing your code, things like ``else`` statements should be the non-normal conditions:
```python
running_normally = ...

if running_normally:
    # do stuff
    ...
else:
    # check what's not running normally...
    ...
```


### **Keep flow blocks as short as reasonably possible.**

``if`` and ``for`` statements should react to a condition and then have a short response block. If the block is taking >10 lines, it should probably be moved to a function. 

```python
too_many_targets_found = ...

if too_many_targets_found:
    # check how many
    # for each item, find its area
    # if the area is very small, remove that target
    # if the perimeter to area ratio is too high it's probably not a real target
elif too_few_targets_found:
    # lower area threshold
    # search for targets again
    ...
```

### **Avoid deep (>3) control flow nests.**

Control flow blocks should not be more than 3 levels deep. Deeply-nested blocks obfuscate code clarity. If such nests are absolutely necessary, move them into a function (abstraction & manage complexity).

Expanding from the above example, the following has deep nests:
```python
targets = ...  # multi-element sequence
too_many_targets_found = ...  # boolean expression

if too_many_targets_found:
    #check how many
    num_targets = find_targets()
    
    # drop items with small areas and high perimeter-area ratios
    for target in targets:
    
        # if the area is very small, remove that target
        if target.area() < 10:
            if target.is_valid():
                del target
                
        # if the perimeter to area ratio is too high it's probably not a real target
        p2a_ratio = target.perimeter() / target.area()
        if p2a_ratio > 0.3:
            del target
            
elif too_few_targets_found:
    # lower area threshold
    # search for targets again
    ...
```

This section could be refactored to avoid deep nests and manage complexity:
```python
targets = ...
too_many_targets_found = ...

def drop_small_targets(targets):
    """Function to drop small targets from the `targets` sequence."""
    for target in targets:
        target_is_small_valid = target.area() < 10 and target.is_valid()
        if target_is_small_valid:
            del target
    return targets
    
def drop_surfacy_targets(targets):
    """Function to drop targets with high perimeter-area ratios."""
    for target in targets:
        p2a_ratio = target.perimeter() / target.area()
        if p2a_ratio > 0.3:
            del target
    return targets
        

if too_many_targets_found:
    #check how many
    num_targets = find_targets()
    
    # drop small targets
    targets = drop_small_targets(targets)
    
    # if the perimeter to area ratio is too high it's probably not a real target
    targets = drop_surfacy_targets(targets)
        
elif too_few_targets_found:
    # lower area threshold
    # search for targets again
    ...
```



### **Each control flow block should have one purpose**

It is tempting to do lots of things within a loop. Since you already have the item/index, you might as well do X with it, right? Nope.

Each block should accomplish one purpose. This may take several statements to accomplish, but the *purpose* should be clear and singular.

```python
... 

if too_many_targets_found:
    # check how many
    num_targets = find_targets()
    
    # drop small targets
    targets = drop_small_targets(targets)
    
    # if the perimeter to area ratio is too high it's probably not a real target
    targets = drop_surfacy_targets(targets)
    
    # NOT THE SINGULAR PURPOSE OF THIS LOOP
    # cleanup existing targets
    for target in targets:
        target.remove_noise()
        target.get_centroid()
        
elif too_few_targets_found:
    # lower area threshold
    # search for targets again
    ...
```

If multiple purposes must be done within a control block, use two blocks:
```python
if too_many_targets_found:
    # same as above
    ...
    
if too_many_targets_found and not_cleaned:
    # clean up targets
    ...
```

### **Don't mess with the index within ``for`` loops.**

``for`` loops automatically increment the index when the statement has done an iteration. Never change this value within the loop. If execution must be skipped for a loop, explicitly skip it rather than change the index.

For example, if I need to skip the 3rd element of a ``for`` loop for some reason I could try changing the index.
```python 
items = ...  # multi-element sequence

for index, item in enumerate(items):
    if index == 3:
        index += 1
        
    # do stuff
    ...
```

Unfortunately, this will not act as expected. It is better to explicitly break out of a loop than change the index:

```python 
items = ...  # multi-element sequence

for index, item in enumerate(items):
    if index == 3:
        continue  # continue means stop executing the current iteration and move to the next iteration
        
    # otherwise, do stuff
    ...
```

### **Use safety counters with ``while`` loops.**

In a ``while`` loop, the block runs until the condition changes to be false. If the condition never changes the block will never end. To avoid infinite loops, add a safety counter condition.

For example, if I'm iteratively thresholding an image until I find the object I'm looking for, I may never find it and I'll be stuck in an infinite loop:
```python
image = ...
threshold = 0.1
object_found = False

while not object_found:
    # apply a threshold to the image
    image.treshold()
    # try to find the object with certain characteristics
    object_found = find_object(image)
    # bump threshold
    threshold += 0.1
```

It's better to add a safety counter:
```python
image = ...
threshold = 0.1
object_found = False
safety_counter = 0

while not object_found or safety_counter < 30:
    ... # same as above
    safety_counter += 1
    
if safety_counter == 30:
    print('While loop exited on safety counter; object not found')
```

One can also ``break`` out of a ``while`` loop:
```python
image = ...
threshold = 0.1
object_found = False
safety_counter = 0

while not object_found:
    ... # same as above
    safety_counter += 1
    if safety_counter == 30:
        print('While loop exited on safety counter; object not found')
        break
```

If we never find the object at least we didn't fall into an infinite loop!